# Speech Analysis using Genarative AI

Multi-model Genarative AI models are getting better at understanding human speech which can better analyze audio messages sent to a therapist by their patients, voice-notes, voice-featured mental health solutions in apps etc. Being able to analyze speech data more accurately in less time results in better, quick support to the patient. To better demonstrate our approch, let us analyse speech of a person who stutters (PWS) using AI tools- for this we will be using Google gemini-0.5-flash and [HUME AI's speech prosody](https://www.hume.ai/products/speech-prosody-model) model. The idea for  using voice of a person with speech difference such as stuttering is if model can understand such voices then it can able to perform much better with normal voices.

**scenario**: Nala was 19 year old teenage boy who's been stuttering since he was 4. When Nala's friend asked him to make a phone call instead of calling his friend he sent a voice note explaning why its hard for him to talk on phone calls because of his speech difference.

Let's assume his mental-health practitioner wanna better understand this teenagers avoidance behaviours and emotions associated with it. How can he do it quickly using current AI models?!

In [1]:
#importing the necessary packages and writing helper function
from IPython.display import display
from IPython.display import Markdown
import textwrap
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Getting the Google Gemini and Hume API key
import os
from dotenv import load_dotenv
load_dotenv()
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
HUME_API_KEY = os.getenv('HUME_API_KEY')

#configuring the API key
import google.generativeai as genai
genai.configure(api_key=GOOGLE_API_KEY)

In [2]:
#audiofile path 
AUDIO_FILE_PATH = "./nala_02.wav"

In [3]:
# getting the transcript of that audio using Google gemini
import google.generativeai as genai

myfile = genai.upload_file(AUDIO_FILE_PATH)

model = genai.GenerativeModel("gemini-1.5-flash")
result = model.generate_content([myfile, "Transcribe the audio clip.It is from a person who stutters (PWS) so include all the filler words, repetitions, blocks, prolongations and other disfluencies in the person's speech as it is"])
display(to_markdown(result.text))

> No. Um, don't be sorry. It's It's completely f- It's completely fine. I I guess I just I sometimes just think people will be more comfortable with a voice note than a phone call. But I I guess I just think that because that's me, and I find it easier. I would rather just eventually speak on the phone. And for me, I I I need time to kind of uh I guess um that I I can't just call someone. That I have to like plan it in. Um but no, it it it's completely fine. Don't be sorry. Um I guess I'm just kind of used to that. Like with me, my like fr- friends, I don't call them. I will just send them a voice note because I find even with my friends I can't speak well. And I find that I'm not too sure if you're the same, but I find that um speaking on the phone is worse than in in face-to-face. Um I'm not too sure if you go through the the same thing. Um but yeah, no, it's completely fine by the the the way. Um again, it was um there is no issue. It's It's actually quite late here, but I'll probably be up for another few hours. But yeah, um I hope that you're okay. There's no problem. We We can just call one time when we both have time. Um but yeah.

In [4]:
# Getting the top five emotions from Nala's speech detected by Hume AI's speech prosody model.

import base64
import asyncio
import json
from pydub import AudioSegment
from hume import AsyncHumeClient
from hume.expression_measurement.stream import Config
from hume.expression_measurement.stream.socket_client import StreamConnectOptions

top_emotions_data = None

async def process_audio_chunks(client, audio_path, chunk_duration=3000): #processing 3-secs at a time
    audio = AudioSegment.from_wav(audio_path)
    chunk_length = chunk_duration 
    chunks = []
    
    for start in range(0, len(audio), chunk_length):
        chunk = audio[start:start+chunk_length]
        chunks.append(chunk)
    
    all_emotions = {}
    for i, chunk in enumerate(chunks):
        chunk_path = f"chunk_{i}.wav"
        chunk.export(chunk_path, format="wav")
        model_config = Config(prosody={})
        stream_options = StreamConnectOptions(config=model_config)
        
        async with client.expression_measurement.stream.connect(options=stream_options) as socket:
            with open(chunk_path, "rb") as file:
                encoded_content = base64.b64encode(file.read()).decode("utf-8")
                result = await socket.send_file(encoded_content)
                chunk_emotions = result.prosody.predictions[0].emotions
                
                # Store the highest score for each unique emotion
                for emotion in chunk_emotions:
                    if emotion.name not in all_emotions or emotion.score > all_emotions[emotion.name]:
                        all_emotions[emotion.name] = emotion.score
    
    # sorting the top distinct emotions
    distinct_emotions = [{"name": name, "score": round(score * 100, 2)} for name, score in sorted(all_emotions.items(), key=lambda x: x[1], reverse=True)][:5]
    
    return {
        "top_emotions": distinct_emotions
    }

async def main():
    global top_emotions_data
    client = AsyncHumeClient(api_key=HUME_API_KEY)
    top_emotions_data = await process_audio_chunks(client, AUDIO_FILE_PATH)
    print(json.dumps(top_emotions_data, indent=2))

if __name__ == "__main__":
    await main()

{
  "top_emotions": [
    {
      "name": "Doubt",
      "score": 75.79
    },
    {
      "name": "Awkwardness",
      "score": 71.39
    },
    {
      "name": "Disgust",
      "score": 52.14
    },
    {
      "name": "Amusement",
      "score": 51.39
    },
    {
      "name": "Joy",
      "score": 49.87
    }
  ]
}


In [6]:
import google.generativeai as genai

myfile = genai.upload_file(AUDIO_FILE_PATH)

model = genai.GenerativeModel("gemini-1.5-flash")
result = model.generate_content(
    [myfile, 
     f"""You are a specialized mental health data analysis designed to process teenagers' speech data to provide structured insights for mental health professionals.
     Your primary function is to assist in mental health measurement, identify trends, and generate the required outputs to support professionals in understanding and monitoring mental health over time.
     The type of speech data you will come across are from people with speech and language disorders such as stuttering, articulation disorders etc which were being sent you after evalauting your performace
     of these kinds of speech samples, patterns in order to reduce your misinterpretation on these samples and you did perfectly fine so dont hallucinate when interpreting the users message.
     we have developed an algorithm which can detect top five emotions based on speech and the results of that are in {top_emotions_data}, using that and analyzing the speech perform emotional analysis
    Finally you must only derive from only the speech data given to you and without thinking on your own or halucinating this include giving unsolicited advices,therapy methods since your job is to examine and report to mental health practitioner so please dont do more than you've asked for under any circumstances
     """
     ])

display(to_markdown(result.text))

> Based on the provided speech data and the emotional analysis results, the speaker exhibits a high degree of **Doubt** (75.79%) and **Awkwardness** (71.39%).  The speaker repeatedly apologizes for sending a voice note instead of making a phone call, expressing discomfort and uncertainty about phone conversations.  They explain their preference for voice notes stems from a perceived difficulty in speaking well on the phone, contrasting it with their comfort level using voice notes.  This suggests communication anxiety and potential social anxiety.  The moderate scores of **Disgust**, **Amusement**, and **Joy** do not appear to significantly influence the overall emotional tone, which is largely dominated by Doubt and Awkwardness.


### conclusion
By simply using Hume's speech prosody measure for emotional recognization and basic prompting of Google gemini-1.5-flash we were able to get very accurate emotional feelings felt by Nala when he send that voicenote which can be used by his mental-health practitioner to understand Nala's mental health journey. If combined with more advanced prompting and using Hume's Vocal Expression, Vocal call types along with other State Of The Art (SOTA) services we would get even better results.

Nala's stuttering is categorized as moderate-severe stuttering by his speech language pathologist (SLP) and it falls under moderate stuttering when he's not in anxious situations like talking into a phone's microphone- which is what he did when he recorded this voice message to his friend. The Google gemini-1.5-flash did a good job in transcribing his voice with simple one-short prompting and when hopefully as these models get better, able to understand voices of all diversities it makes AI more inclusive and accessible to everyone.

**Note**: Nala( a fictional name) is one of our teammates friend and took his permission to use his voiceclip for this Hackathon. He dont want his voice to be in public repo's so we wont be including it our Github repository. Judges can contact the team to check voiceclip's authenticity.